In [2]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from keras.models import load_model

import numpy as np
import librosa
import librosa.display
import sklearn
import matplotlib.pyplot as plt
import speech_recognition as VTT
import os
import sys

In [3]:
DATA_DIR = '1. VoiceToEmotion/Data/Sample'

In [4]:
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))

Voice To Emotion _ 곽민창

In [5]:
#Audio = 원본 통화녹음 파일 Data/Sample/Sample1.wav

audio, sr = librosa.load(DATA_DIR + '/Sample1.wav', sr=None)
print('sr : ', sr)
print('wav shape : ', audio.shape)
print('length : ', audio.shape[0]/float(sr), 'secs')

sr :  48000
wav shape :  (252720,)
length :  5.265 secs


In [6]:
mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=100, n_fft=400, hop_length=160)
mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
padded_mfcc = pad2d(mfcc, 700)

C:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\librosa\util\decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
C:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
C:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


In [7]:
padded_mfcc = np.expand_dims(padded_mfcc, 0)

In [9]:
model = load_model('weight.h5')
VTE_result = model.predict(padded_mfcc)

1/1 [==============================] - 0s 207ms/step


In [10]:
VTE_result

array([[0.9997571, 0.9709723]], dtype=float32)

Voice To Text _ 이승렬

In [11]:
r = VTT.Recognizer()
kr_audio = VTT.AudioFile(DATA_DIR + '/Sample1.wav')

In [12]:
name = 0

In [13]:
with kr_audio as source:
    VTT_audio = r.record(source)

In [14]:
temp = r.recognize_google(VTT_audio, language='ko-KR')

result2:
{   'alternative': [   {   'confidence': 0.82105619,
                           'transcript': '얼굴은 화끈 화끈 하고 가슴이 무서워 하는 사람처럼 뛰어 '
                                         '놀았습니다'},
                       {'transcript': '얼굴은 화끈 화끈 하고 가슴이 무서워하는 사람처럼 뛰어 놀았습니다'},
                       {'transcript': '얼굴은 화끈화끈 하고 가슴이 무서워 하는 사람처럼 뛰어 놀았습니다'},
                       {'transcript': '얼굴은 화끈화끈 하고 가슴이 무서워하는 사람처럼 뛰어 놀았습니다'},
                       {'transcript': '얼굴은 화끈 화끈 하고 가슴이 무서워 하는 사람처럼 띠 놀았습니다'}],
    'final': True}


In [15]:
sys.stdout = open(str(name) + '.txt', 'w')
print(temp)
#sys.stdout.close()
name = name + 1

Text To Emotion _ 김지호

In [ ]:
import nltk
from konlpy.tag import Okt
import json
okt = Okt()

In [16]:
def read_data(filename):
    with open(filename, 'r', encoding = 'UTF8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

In [17]:
def tokenize(doc):
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [18]:
train_data = read_data("2. kimjiho/문자열_train_data.txt")
test_data = read_data("2. kimjiho/문자열_test_data.txt")

In [ ]:
if os.path.isfile('train_docs.json'):
    with open('train_docs.json',encoding="UTF-8")as f:
        train_docs = json.load(f)
    with open('test_docs.json',encoding="UTF-8")as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row[0]),row[1]) for row in train_data]
    test_docs = [(tokenize(row[0]),row[1]) for row in test_data]
    with open('train_docs.json','w',encoding="UTF-8")as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('test_docs.json','w',encoding="UTF-8")as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

In [ ]:
tokens = [t for d in train_docs for t in d[0]]
text = nltk.Text(tokens, name='NMSC')
selected_words = [f[0] for f in text.vocab().most_common(20000)]
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

In [ ]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.array(tf).astype('float32'),axis=0)
    score = float(model.predict(data))
    if(score>0.5):
        print("오늘의 긍정지수는 {:2f}%입니다.\n".format(score*100))
        TTE_Result = score*100
    else:
        print("오늘의 부정지수는 {:2f}%입니다.\n".format((1-score)*100))
        TTE_Result = score*100

In [ ]:
predict_pos_neg("input 입력할것")